In [ ]:
from data import get_loader, test_dataset
from options import opt
import torch.backends.cudnn as cudnn
import logging
from tensorboardX import SummaryWriter
from utils import clip_gradient, adjust_lr
from models.BBSNet_model import BBSNet
from torchvision.utils import make_grid
from datetime import datetime
import numpy as np
import os
import torch
import torch.nn.functional as F
import sys
sys.path.append('./models')

from BBSNet_train import *

%load_ext autoreload
%autoreload 2

In [ ]:
setup_gpu()
model, optimizer, CE = get_model()
train_loader, test_loader, total_step = get_data()
main(optimizer, model, train_loader, test_loader, save_path)